In [ ]:
from cling import cling, bash

**Listing 9.1**

Caption: Example for sequential program execution and asynchronous program execution for the Ultimate Question of Life, the Universe, and Everything.

In [ ]:
%%cling
#include <iostream>
#include <hpx/hpx.hpp>
//-----------------------------------------------------------------
auto compute_answer = []() -> int {
  // Compute the Ultimate Question of Life,
  // the Universe, and Everything.
  return 42;
};
auto compute_question = []() -> std::string {
  // compute the actual 'Ultimate Question'
  // which goes with the answer.
  return "What do you get when multiply 6 by 9?";
};

// Sequential execution
int answer = compute_answer();
// Wait for 7.5 million years...
std::string question = compute_question();
std::cout << "answer: " << answer << " question: " << question << std::endl;
//-----------------------------------------------------------------
{ // Thread and join need to be in one execution block
// Low-Level thread execution
hpx::thread answer_thread([&](){
    answer = compute_answer();
});
// While waiting for 7.5 million years...
question = compute_question(); // concurrent with compute_question
answer_thread.join();
std::cout << "answer2: " << answer << " question2: " << question << std::endl;
}
//-----------------------------------------------------------------
// Asynchronous execution
hpx::future<int> f = hpx::async(compute_answer);  
// While waiting for 7.5 million years...
question = compute_question();
auto result = f.get();  
std::cout << "answer3: " << result << " question3: " << question << std::endl;


**Listing 9.2**

Caption: Parallel implementation of the natural logarithm using \cpp{hpx::async} and \cpp{hpx::future}.

In [ ]:
%%cling
#include <pbm.hpp>
#include <config.hpp>
#include <kernel.hpp>

#include <algorithm>
#include <iostream>
#include <numeric>
#include <hpx/hpx.hpp>
//-----------------------------------------------------------------
const size_t partition_size = 25;
const size_t num_partitions = 4;
std::vector<double> parts(partition_size*num_partitions);
std::iota(parts.begin(), parts.end(), 1);
const size_t partitions = 4;
std::vector<hpx::future<double>> futures; 
double x = 0.1;
for (size_t i = 0; i < num_partitions; i++) {
  size_t begin = i * partition_size;
  size_t end = (i + 1) * partition_size;

  hpx::future<double> f = hpx::async([begin, end]() -> double {
    std::for_each(parts.begin() + begin, parts.begin() + end, [](double& e) {
      e = std::pow(-1.0, e + 1) * std::pow(x, e) / (e);
    });

    return std::accumulate(parts.begin() + begin, parts.begin() + end, 0.); 
  });

  futures.push_back(std::move(f));
}

double result = 0;
for (size_t i = 0; i < futures.size(); i++) result += futures[i].get(); 
std::cout << result << std::endl;


**Listing 9.3**

Caption: Example for sequential program execution and asynchronous program execution for the Ultimate Question of Life, the Universe, and Everything.

In [ ]:
%%cling
#include <iostream>
#include <hpx/hpx.hpp>
//-----------------------------------------------------------------
auto compute_answer = []() -> int {
  // Compute the Ultimate Question of Life,
  // the Universe, and Everything.
  return 42;
};
auto compute_question = []() -> std::string {
  // compute the actual 'Ultimate Question'
  // which goes with the answer.
  return "What do you get when multiply 6 by 9?";
};
hpx::future<int> result;
std::string question;

result  = hpx::async(compute_answer);
// While waiting for 7.5 million years...
question = compute_question();
std::cout << "Before get" << std::endl;
std::cout << "answer: " << result.get() << " question: " << question << std::endl;
std::cout << "After get" << std::endl;

std::cout << "======================" << std::endl;

// Asynchronous execution
result  = hpx::async(compute_answer);
// While waiting for 7.5 million years...
question = compute_question();
hpx::future<void> done = result.then([](hpx::future<int> result_) { 
    std::cout << "answer: " << result_.get() << " question: " << question << std::endl; 
});
sleep(1); // give the print time to happen
std::cout << "Before get" << std::endl;
done.get();
std::cout << "After get" << std::endl;


**Listing 9.4**

Caption: Parallel implementation of the natural logarithm using \cpp{hpx::async}, \cpp{hpx::future}, and \cpp{hpx::when_all}.

In [ ]:
%%cling
#include <pbm.hpp>
#include <config.hpp>
#include <kernel.hpp>

#include <hpx/future.hpp>
#include <hpx/hpx.hpp>
#include <hpx/numeric.hpp>
#include <iostream>
//-----------------------------------------------------------------
const size_t num_partitions = 10;
const size_t partition_size = 25;
const size_t n = num_partitions*partition_size;
std::vector<double> parts(n);
std::iota(parts.begin(), parts.end(), 1);

double x=0.1;

std::vector<hpx::future<double>> futures;
for (size_t i = 0; i < num_partitions; i++) {
  size_t begin = i * partition_size;
  size_t end = (i + 1) * partition_size;

  hpx::future<double> f = hpx::async([begin, end]() -> double {
    std::for_each(parts.begin() + begin, parts.begin() + end, [](double& e) {
      e = std::pow(-1.0, e + 1) * std::pow(x, e) / (e);
    });

    return hpx::reduce(parts.begin() + begin, parts.begin() + end, 0.);
  });

  futures.push_back(std::move(f));
}

double result = hpx::when_all(futures) 
    .then([](auto&& f) { 
      auto futures = f.get(); 

      double result = 0;
      for (size_t i = 0; i < futures.size(); i++)
        result += futures[i].get();
      return result;
    })
    .get();   

std::cout << "Difference of Taylor and C++ result " << result - std::log1p(x)
          << " after " << n << " iterations." << std::endl; 


**Listing 9.5**

Caption: Example for sequential program execution and asynchronous program execution for the Ultimate Question of Life, the Universe, and Everything.

In [ ]:
%%cling
#include <iostream>
#include <hpx/hpx.hpp>
//-----------------------------------------------------------------
auto compute_answer = []() -> int {
  // Compute the Ultimate Question of Life,
  // the Universe, and Everything.
  return 42;
};
auto compute_question = []() -> std::string {
  // compute the actual 'Ultimate Question'
  // which goes with the answer.
  return "What do you get when multiply 6 by 9?";
};
hpx::future<int> answer;
hpx::future<std::string> question;

answer  = hpx::async(compute_answer);
question = hpx::async(compute_question);
hpx::future<void> result = hpx::dataflow([](hpx::future<int> a,hpx::future<std::string> q) {
    std::cout << "Q: " << q.get() << " A: " << a.get() << std::endl;
}, answer, question);
sleep(1);

std::cout << "Before get" << std::endl;
result.get();
std::cout << "After get" << std::endl;


**Listing 9.6**

Caption: Example for sequential program execution and asynchronous program execution for the Ultimate Question of Life, the Universe, and Everything.

In [ ]:
%%cling
#include <iostream>
#include <hpx/hpx.hpp>
//-----------------------------------------------------------------
auto compute_answer = []() -> int {
  // Compute the Ultimate Question of Life,
  // the Universe, and Everything.
  return 42;
};
auto compute_question = []() -> std::string {
  // compute the actual 'Ultimate Question'
  // which goes with the answer.
  return "What do you get when multiply 6 by 9?";
};
hpx::future<int> answer;
hpx::future<std::string> question;

answer  = hpx::async(compute_answer);
question = hpx::async(compute_question);
hpx::future<void> result = hpx::dataflow(hpx::unwrapping(
    [](int a,std::string q) { 
    std::cout << "Q: " << q << " A: " << a << std::endl;
}), answer, question);
sleep(1);

std::cout << "Before get" << std::endl;
result.get();
std::cout << "After get" << std::endl;


**Listing 9.7**

Caption: A \cpp{hpx::future} which is ready immediately by using \cpp{hpx::make_ready_future}.

In [ ]:
%%cling
#include <iostream>
#include <hpx/future.hpp>
//-----------------------------------------------------------------

auto f = hpx::make_ready_future(1);
/*
* Since the future is ready the output will happen
* without any waiting.
*/
std::cout << f.get() << std::endl;


**Listing 9.8**

Caption: An asynchronous fractal code that can be compiled and run for either HPX or the standard library.

In [ ]:
%%writefile main_async_benchmark.cpp
#define USE_HPX 1
#include <algorithm>
#include <future>
#include <iostream>
#include <numeric>
#include <pbm.hpp>
#include "config.hpp"
#include "kernel.hpp"
//-----------------------------------------------------------------
#if USE_HPX 
#include <hpx/hpx.hpp>
#include <hpx/hpx_main.hpp>
namespace par = hpx;
#else
namespace par = std;
#endif

void launch(size_t begin, size_t end, PBM* pbm) {
  for (size_t i = begin; i < end; i++) {
    complex c = complex(0, 4) * complex(i, 0) /
        complex(size_x, 0) - complex(0, 2);

    for (size_t j = 0; j < size_y; j++) {
      int value = compute_pixel(c + 4.0 * j / size_y - 2.0);
      // Convert the value to RGB color space
      std::tuple<size_t, size_t, size_t> color = get_rgb(value);
      // Set the pixel color
      (*pbm)(i, j) = make_color(std::get<0>(color),
                                std::get<1>(color),
                                std::get<2>(color));
    }
  }
}

int main(int argc, char* argv[]) {
  size_t partitions = get_size_t("NUM_THREADS", 3);
  size_t output = get_size_t("OUTPUT", 1);
  PBM pbm = PBM(size_x, size_y);

  size_t size = std::round(size_x / partitions);
  std::vector<par::future<void>> futures; 
  auto start = std::chrono::high_resolution_clock::now();
  for (size_t i = 0; i < partitions; i++) {
    size_t start = i * size;
    size_t end = (i + 1) * size;
    if (i == partitions - 1) end = size_x;
    auto f = par::async(launch, start, end, &pbm); 
    futures.push_back(std::move(f)); 
  }
  for (auto&& f : futures) f.wait(); 
  auto stop = std::chrono::high_resolution_clock::now();
  auto duration =
      std::chrono::duration_cast<std::chrono::microseconds>(stop - start);
  std::cout << duration.count() * 1e-6 << std::endl;

  // Save the image
  if (output == 1) pbm.save("image_future_parallel_" + type + ".pbm");

  return EXIT_SUCCESS;
}


In [ ]:
%%bash
hpxcxx -I . --exe=main_async_benchmark.exe main_async_benchmark.cpp
hpxrun.py -l 3 -t 1 ./main_async_benchmark.exe